In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

In [ ]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset

# Step 1: Download the JSON file
url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
response = requests.get(url)
json_data = response.json()


In [ ]:
def json_to_dataframe(json_data):
    data = []
    for document in json_data:
        doc_id = document.get("id")
        for annotation in document.get("annotations", []):
            for result in annotation.get("result", []):
                segment = {
                    'doc_id': doc_id,
                    'text': result['value'].get('text'),
                    'label': result['value'].get('labels', [None])[0]  # Get the first label if available
                }
                data.append(segment)
    return pd.DataFrame(data)

In [ ]:
# Convert JSON to DataFrame
df = json_to_dataframe(json_data)
print(df.head(100))

    doc_id                                               text     label
0     1735        IN THE HIGH COURT OF KARNATAKA,\n       ...  PREAMBLE
1     1735  \n\n      BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...  PREAMBLE
2     1735  This Criminal Appeal is filed under Section 37...  PREAMBLE
3     1735  \n\n       This appeal coming on for hearing t...  PREAMBLE
4     1735  \n       Heard the learned Counsel for the app...      NONE
..     ...                                                ...       ...
95    4183                                          \nJUSTICE  PREAMBLE
96    4183  .\n\n         THIS I.T.A. COMING ON FOR HEARIN...  PREAMBLE
97    4183          \nS. SUJATHA J. DELIVERED THE FOLLOWING:-  PREAMBLE
98    4183       \n\n                                JUDGMENT  PREAMBLE
99    4183  \nMr. E.I.Sanmathi Adv. for Appellants- Revenu...      NONE

[100 rows x 3 columns]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)
# Convert DataFrame to Hugging Face Dataset again
dataset = Dataset.from_pandas(df)

{'ANALYSIS': 0, 'ARG_PETITIONER': 1, 'ARG_RESPONDENT': 2, 'FAC': 3, 'ISSUE': 4, 'NONE': 5, 'PREAMBLE': 6, 'PRE_NOT_RELIED': 7, 'PRE_RELIED': 8, 'RATIO': 9, 'RLC': 10, 'RPC': 11, 'STA': 12}


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from transformers import AutoModel


In [ ]:
# Load the tokenizer and model
model_name = "Saibo-creator/legal-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['label'])))
# Step 2: Convert DataFrame to Hugging Face Dataset and encode labels
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
"""
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(df)"""

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at Saibo-creator/legal-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/28986 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
# Split the dataset into train, test, and eval sets
# Adjust the splits according to your needs
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_test_dataset = split_dataset['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': split_dataset['train'],
    'test': train_test_dataset['test'],
    'eval': train_test_dataset['train']
})

In [ ]:
# Step 3: Setup Training Arguments
training_args = TrainingArguments(
    output_dir="./testingresultsLegalRo-BERTagain",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_total_limit=1,
    save_steps=100,
    logging_dir="./logs",
    report_to="wandb"  # Ensure W&B is properly set up, or remove this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['eval']
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.181700
1000,0.881900
1500,0.772300


# Precision, Recall, F1 score, Accuracy (Evaluating Model)

eval set testing

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# If the model is already trained, just initialize the Trainer with the same model
trainer = Trainer(
    model=model,  # Use your already trained model
    args=training_args,
    eval_dataset=datasets['eval'],  # Use your evaluation dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(results)


test set testing

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize the Trainer with the already trained model
trainer = Trainer(
    model=model,  # Use your already trained model
    args=training_args,
    eval_dataset=datasets['test'],  # Use your test dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print("Test Set Evaluation Results:")
print(results)


Confusion matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from transformers import Trainer

# Define a function to compute predictions on the test set
def get_predictions(trainer, test_dataset):
    predictions = trainer.predict(test_dataset)
    preds = predictions.predictions.argmax(axis=-1)  # Get the predicted labels
    labels = predictions.label_ids  # Get the true labels
    return preds, labels

# Get predictions from the model
preds, labels = get_predictions(trainer, datasets['test'])

# Compute the confusion matrix
conf_matrix = confusion_matrix(labels, preds)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Make predictions
predictions = trainer.predict(tokenized_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Print predicted labels
print(predicted_labels)

KeyboardInterrupt: 

# save this model on huggingface

In [ ]:
pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from huggingface_hub import HfApi

# Define repository name and create repo on the hub
repo_name = "LegalRo-BERt_for_rhetorical_role_labeling"  # Choose a unique name
HfApi().create_repo(repo_name)

# Upload the model
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/engineersaloni159/LegalRo-BERt_for_rhetorical_role_labeling/commit/26b6f6cbad6d06cdc827f55c637495bc3c70ec18', commit_message='Upload tokenizer', commit_description='', oid='26b6f6cbad6d06cdc827f55c637495bc3c70ec18', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained(repo_name, push_to_hub=True)
tokenizer.save_pretrained(repo_name, push_to_hub=True)

('LegalRo-BERt_for_rhetorical_role_labeling/tokenizer_config.json',
 'LegalRo-BERt_for_rhetorical_role_labeling/special_tokens_map.json',
 'LegalRo-BERt_for_rhetorical_role_labeling/vocab.json',
 'LegalRo-BERt_for_rhetorical_role_labeling/merges.txt',
 'LegalRo-BERt_for_rhetorical_role_labeling/added_tokens.json',
 'LegalRo-BERt_for_rhetorical_role_labeling/tokenizer.json')

# Manually Evaluation of model

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121


In [ ]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import load_dataset, Dataset
import pandas as pd
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("engineersaloni159/LegalRo-BERt_for_rhetorical_role_labeling")
model = AutoModelForSequenceClassification.from_pretrained("engineersaloni159/LegalRo-BERt_for_rhetorical_role_labeling")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Ensure model is in evaluation mode
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
#run this cell if model is directly called from  huggingface
from sklearn.preprocessing import LabelEncoder

# Assuming you already have this
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Print the mapping of labels to numbers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'ANALYSIS': 0, 'ARG_PETITIONER': 1, 'ARG_RESPONDENT': 2, 'FAC': 3, 'ISSUE': 4, 'NONE': 5, 'PREAMBLE': 6, 'PRE_NOT_RELIED': 7, 'PRE_RELIED': 8, 'RATIO': 9, 'RLC': 10, 'RPC': 11, 'STA': 12}


In [ ]:
# Assuming label_encoder is the one you used for encoding labels earlier

# Example sentence
sentence =  "This Court quoted. with approval the decision in Commissioner of Excess Profits Tax, Bombay City v. Bhogilal H. Patel Bombay (2 ) and held that the language used in the third proviso to S. 5 of the Excess Profits Tax Act, 1940 was one of exclusion and that Act did not apply to profits etc. of that part of the business which arose in an Indian State. "
# Tokenize the sentence
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# The predictions are in the form of logits, you can convert them to probabilities
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)

# Convert the predicted class number to the original label
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: PRE_RELIED


In [ ]:
print("Logits:", logits)


Logits: tensor([[ 2.2846,  0.2215, -0.3713, -0.5981, -0.7258, -0.8616,  0.2079, -1.1513,
          0.6050, -0.1885, -0.6332, -0.5202, -0.2322]])


predict RR wih OpenAyAI

In [ ]:
pip install opennyai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This could take

In [ ]:
from opennyai import RhetoricalRolePredictor
from opennyai.utils import Data
import urllib
from opennyai import Pipeline


In [ ]:
###### Get court judgment texts on which to run the AI models
text1 = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment1.txt').read().decode()
text2 = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment2.txt').read().decode()
texts_to_process = [text1,text2] ### you can also load your text files directly into this
data = Data(texts_to_process)  #### create Data object for data  preprocessing before running ML models
use_gpu = False
pipeline = Pipeline(components=['Rhetorical_Role'], use_gpu=use_gpu, verbose=True)

results = pipeline(data)

json_result_doc_1 = results[0]

ℹ Installing en_core_web_trf. This is a one time process!!
ℹ Pre-processing will happen on GPU!
ℹ Loading Rhetorical Role...
ℹ Rhetorical Roles will use CPU!


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: "https://huggingface.co/opennyaiorg/InRhetoricalRoles/resolve/main/InRhetoricalRoleModel.pt" to /root/.opennyai/rhetoricalrole/InRhetoricalRol

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

ℹ Preprocessing rhetorical role model input!!!


100%|██████████| 2/2 [00:12<00:00,  6.01s/it]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ℹ Processing documents with rhetorical role model!!!


100%|██████████| 2/2 [01:42<00:00, 51.34s/it]


In [ ]:
json_result_doc_1

{'id': '7411d72c470a731a669272d809fce542010776c9bd0885112fb9a9217522f5da',
 'data': {'text': '                                                   Reportable\n\n            In The Supreme Court Of India\n             Civil Appellate Jurisdiction \n\n             Civil Appeal No. 8990 Of 2017 \n\nExecutive Engineer (R And B)\nAnd Others                                    ... Appellant(S) \n\n                           Versus \nGokul Chandra Kanungo (Dead) \nThr. His Lrs.                                ... Respondent(S) \n\n                         Judgment\nB.R. GAVAI, J.\n\n1. The appellants have challenged the judgment dated 18th April 2012 passed by the learned Single Judge of the High Court of Orissa in Arbitration Appeal No. 25 of 2007, thereby dismissing the appeal filed by the appellants. \n\n2. The facts in brief giving rise to the present appeal are as under:\n\nThe respondent was awarded the contract for construction of 3 kilometers missing link on NH6 from Kanjipani to Kuntala 

In [ ]:
text3="""
JUDGMENT
Indira Banerjee, J.
Leave granted.
This appeal filed by the Original Plaintiff, Chandrabhan,  since deceased, represented by his legal
representatives , is against a final judgment and order dated 11 th January 2016 passed by the
Aurangabad Bench of the High Court of Judicature at Bombay, allowing Second Appeal No.
45 of
1995 filed by the Respondents, reversing the judgment and order dated 10th November 1994 passed
by the Additional District Judge, Beed, in Regular Civil Appeal No.361 of 1984 and dismissing
Regular Civil Suit No.
198 of 1979 filed by the Original Plaintiff praying for the relief of declaration
of ownership and perpetual injunction in respect of the suit property.
Sambhaji, grandfather of the Original Plaintiff, Chandrabhan had two sons, Baliram and
Rambhau.
Baliram was the elder of the two sons of Sambhaji.
The Original Defendant No.1
Yamunabai, was the wife of Baliram.
September, 2022
Baliram however had two sons, Digamber and Chandrabhan  Original Plaintiff .
Since Baliram and Yamunabai were childless, Baliram decided to adopt his nephew, Chandrabhan
 Original Plaintiff .
Chandrabhan  Original Plaintiff  was Balirams younger brother Rambhaus son,
as noted above.
It is stated that Baliram and Rambhau had mutually agreed that Baliram would
adopt Rambhaus son, Chandrabhan  Original Plaintiff .
In the plaint, it was pleaded that the Original Plaintiff, Chandrabhan was adopted by his uncle
Baliram, in accordance with the rites and customs of the community, in a ceremony attended by
relatives, neighbours and friends.
According to the Appellants, the Original Plaintiff Chandrabhan
was about 14 years of age at the time of his adoption by his paternal uncle Baliram, who became his
adoptive father.
Baliram died intestate, in 1951, about six months after he adopted the Original Plaintiff.
After the
death of his adoptive father Baliram, the Original Plaintiff Chandrabhan shifted to a nearby village,
from where he managed the properties left by Baliram.
The Original Defendant No.2 Champabai is
the first wife of the Original Plaintiff, Chandrabhan.
In 1979, the Original Defendant No.
1,
Yamunabai, wife of Baliram purportedly gifted the suit properties to the Original Defendant No.
2
Champabai, wife of the Original Plaintiff.
On 8th May 1979, the Original Plaintiff filed Regular Civil Suit No.
198 of 1979, in the Court of the
Civil Judge, Senior Division, at Beed, Maharashtra against the Original Defendant No.1 being
Yamunabai, the wife of Baliram and the Original Defendant No.2 being Champabai, his own first
wife praying for declaration of ownership of the suit properties, perpetual injunction and other
reliefs.
The Original Defendants filed their written statements in the suit contending that the
Original Plaintiff had not been adopted by Baliram.
The Respondent Nos.
1 to 4 being the Original Defendant Nos.
3 to 6 in the suit, filed their written
statement in the suit supporting the stand of the Original Defendant Nos.
1 and 2.
The Original Plaintiff examined himself as well as five other witnesses, including Prabhu Yogiraj
Swami, the priest who conducted the rituals at the time of adoption, to prove that he had been
adopted by Baliram.
The Original Plaintiff also examined Shahurao Tulsiram Dhas to prove that he
had the possession and cultivation of the suit lands.
The Respondents, on the other hand, examined
Original Defendant No.2, Champabai and several others to establish that the Original Plaintiff had
not been legally and or validly adopted by Baliram.
The Civil Judge, Junior Division, Beed, Maharashtra being the Trial Court dismissed the Regular
Civil Suit No.
198 of 1979, by a judgment and order dated 31st July 1984.
On 5th November 1984, the Original Plaintiff filed an appeal being Regular Civil Appeal No.361
of 1984 in the Court of the Additional District Judge, Beed, Maharashtra being the First Appellate
Court.
By a judgment and order dated 10th November 1994, the First Appellate Court allowed the September, 2022
the Trial Court whereby Regular Civil Suit No.198 of 1979 had been dismissed.
The First Appellate Court, after considering the evidence on record, concluded that the Original
Plaintiff had been adopted by Baliram and thus entitled to succeed the property of Baliram after his
death.
The Respondent Nos.1 to 4, being the Original Defendant Nos.
3 to 6, in the suit, who were
purchasers pendente lite of the suit property filed a Second Appeal No.45 of 1995 in the High Court
of Judicature at Bombay  Aurangabad Bench .
The Original Defendant No.1 and the Original
Defendant No.2 did not challenge the order passed by the First Appellate Court.
The High Court admitted the Second Appeal, which was heard at length and allowed by a
judgment and order dated 11 th January 2016, which is impugned in this appeal before us.
The High Court considered the following questions.
Whether the first appellate Court has committed error in not considering the circumstance that
other transactions of sale made by the defendant No.1 in respect of three agricultural lands like
Survey Nos.86/1, 100/3 and 109/2 which were left behind by Baliram are not challenged by the
plaintiff in the suit?.
II  Whether the first appellate Court has committed error in not considering the circumstance that
after the death of Baliram name of defendant No.1 only was mutated in the revenue record as
successor of Baliram and the name of the plaintiff was not entered as successor of Baliram?.
III  Whether the first appellate Court has committed error in not considering the circumstance that
the cooperative credit society could not have given loan to the plaintiff on the lands left behind by
Baliram as plaintiff was not shown as owner in the revenue record and further there is the
circumstance that it is defendant No.1 who had repaid the loan?.
IV  Whether the first appellate Court has committed error in not giving due weight to the
circumstance like plaintiff never used name of Baliram as his father anywhere and he continued to
use the name of his natural father Rambhau?.
Whether due to absence of specific pleadings with regard to particulars of adoption and due to
inconsistencies in the evidence of the witnesses it can be said that there is sufficient evidence to
prove the factum of adoption?.
We find there were no questions of law before the High Court, not to speak of substantial
questions of law.
Admittedly, evidence was adduced at the trial.
The Original Plaintiff examined himself as witness
and examined five other witnesses.
The Original Plaintiff gave evidence of adoption.
The second September, 2022
there was an adoption ceremony held at the residence of Rambhau.
She also stated that the Original
Plaintiff Chandrabhan had performed the last rites of Baliram as his adopted son.
The other
witnesses also deposed that they had attended the ceremony at which the Original Plaintiff
Chandrabhan had been given in adoption to Baliram.
The Trial Court rejected the contention of the
Plaintiff on the ground of contradiction and inconsistencies in the evidence.
The Trial Court also found that the performance of the essential requisites of adoption, such as
giving in adoption and taking in adoption had not been established.
The First Appellate Court re-analysed the evidence and found that some discrepancies and
inconsistencies were natural since the adoption had taken place in 1950 and evidence was taken in
1984, about 34 years later.
Thereby, some inconsistencies were only natural.
The finding of the First Appellate Court that the Original Plaintiff, Chandrabhan was the adopted
son of Baliram was based on -

The evidence of Trivenibai  PW-2 , wife of the Original Plaintiff, Chandrabhans
elder brother Digambar,

The evidence of the priest, Prabhu Yogiraj Swami who conducted the ceremony of
adoption.
PW-6 .
The admission that PW-6 was the family priest who performed rituals of the
community to which the parties belonged.
Evidence that the Original Plaintiff, Chandrabhan had been residing in the house
of Baliram.
The Original Plaintiff Chandrabhans name shown as Chandrabhan Baliram in
registers and documents dating back 1960-61.
The fact that Champabai, the first wife of the Original Plaintiff was residing with
Original Defendant No.1, Yamunabhai.
The properties of Rambhau were inherited by Digamber alone the Original
Plaintiff did not get any share in the properties.
Evidence of PW-7, Bansi Hajare who had been Secretary of the Ghargaon
society for the period from 1961 to 1963, mentioned that there was a crop loan
account of Chandrabhan Baliram by Sl.
No.35 in his register.
The register shows that
Chandrabhan Baliram repaid loan of Rs.150/- by 31st July 1961.
September, 2022
reside with the Original Plaintiff as stated in her evidence.
Champabai had stated that the Original
Plaintiff, Shivganga and her were residing together.
The use of the middle name, which is the
fathers name, as Baliram instead of Rambhau after Chandrabhan, gave rise to inference of adoption
of Chandrabhan by Baliram.
It is well settled that a Second Appeal under Section 100 of the Civil Procedure Code, 1908  CPC
can only be entertained on a substantial question of law.
In H.P.
Pyarejan v. Dasappa  Dead  by LRs.
and Others1, this Court held-

In our opinion, therefore, the judgment of the High Court suffers from serious
infirmities.
It suffers from the vice of exercise of jurisdiction which did not vest in the
High Court under the law.
Under Section 100 of the Code  as amended in 1976  the
jurisdiction of the High Court to interfere with the judgments of the courts below is
confined to hearing on substantial questions of law.
Interference with finding of fact
by the High Court is not warranted if it involves reappreciation of evidence  see
Panchugopal Barua v. Umesh Chandra Goswami  1997  4 SCC 713 and Kshitish
Chandra Purkait v. Santosh Kumar Purkait  1997  5 SCC 438  .
The High Court has
not even discussed any evidence.
No basic finding of fact recorded by the courts
below has been reversed much less any reason assigned for taking a view contrary to
that taken by the courts below.
The finding on the question of readiness and 1  2006
2 SCC 496 willingness to perform the contract which is a mixed question of law and
fact has been upset.
It is statutorily provided by Section 16 1  c  of the Act that to
succeed in a suit for specific performance of a contract the plaintiff shall aver and
prove that he has performed and has always been ready and willing to perform the
essential terms of the contract which were to be performed by him other than the
terms the performance of which has been prevented or waived by the defendant.
In Ram Prasad Rajak v. Nand Kumar Bros. and Another2, this Court held that, Once the
proceeding in the High Court is treated as a Second Appeal under Section 100 CPC, the restrictions
prescribed in the said Section would come into play.
The High Court could and ought to have dealt
with the matter as a Second Appeal and found out whether a substantial question of law arose for
consideration.
Unless there was a substantial question of law, the High Court had no jurisdiction to
entertain the Second Appeal and consider the merits.
In Kshitish Chandra Purkait v. Santosh Kumar Purkait and Others 3, this Court held that
existence of substantial question of law was the sine qua non for the exercise of jurisdiction under
Section 100 of the CPC.
In Kshitish Chandra Purkait  supra , this Court held-

We would only add that  a  it is the duty cast upon the High Court to formulate
the substantial question of law involved in the case even at the initial stage and  b
that in  exceptional  cases, at a later point of time, when the Court exercises its September, 2022
the substantial question of law, the opposite party should be put on notice thereon
and should be given a fair or proper opportunity to meet the point.
Proceeding to
hear the appeal without formulating the substantial question of law involved in the
appeal is illegal and is 2  1998  6 SCC 748 3  1997  5 SCC 438 an abnegation or
abdication of the duty cast on court and even after the formulation of the substantial
question of law, if a fair or proper opportunity is not afforded to the opposite side, it
will amount to denial of natural justice.
The above parameters within which the High
Court has to exercise its jurisdiction under Section 100 CPC should always be borne
in mind.
We are sorry to state that the above aspects are seldom borne in mind in
many cases and second appeals are entertained and or disposed of, without
conforming to the above discipline.
The guidelines to determine what is a substantial question of law within the meaning of Section
100 CPC has been laid down by this Court in Sir Chunnilal V. Lal Mehta Sons v. Century Spinning
and Manufacturing Co. Ltd.4

In Sir Chunilal V. Mehta and Sons  supra , this Court agreed with and approved a Full Bench
judgment of the Madras High Court in Rimmalapudi Subba Rao v. Noony Veeraju and Ors.
5 which
laid down the principles for deciding when a question of law becomes a substantial question of law.
In Hero Vinoth v. Seshammal6, this Court followed Sir Chunilal v. Mehta Sons  supra  and
other judgments and summarized the tests to find out whether a given set of questions of law were
mere questions of law or substantial questions of law.
The relevant paragraphs of the judgment of this Court in Hero Vinoth  supra  are set out herein
below

The phrase substantial question of law, as occurring in the amended Section
100 CPC is not defined in the Code.
The word substantial, as qualifying question of
law, means of having substance, essential, real, of sound worth, important or 4 AIR
1962 SC 1314 5 AIR 1951 Mad 969 6  2006  5 SCC 545 considerable.
It is to be
understood as something in contradistinction with technical, of no substance or
consequence, or academic merely.
However, it is clear that the legislature has chosen
not to qualify the scope of substantial question of law by suffixing the words of
general importance as has been done in many other provisions such as Section 109
of the Code or Article 133 1  a  of the Constitution.
The substantial question of law on
which a second appeal shall be heard need not necessarily be a substantial question
of law of general importance.
In Guran Ditta v.
Ram Ditta 55IA 235 AIR 1928 PC 172 the phrase substantial question of law as it was employed
in the last clause of the then existing Section 100 CPC  since omitted by the Amendment Act, 1973
came up for consideration and their Lordships held that it did not mean a substantial question of
general importance but a substantial question of law which was involved in the case.
In Sir Chunilal September, 2022
the following view taken by a Full Bench of the Madras High Court in Rimmalapudi Subba Rao v.
Noony Veeraju AIR 1951 Mad 969  1951  2 MLJ 222  FB   Sir Chunilal case 1962 Supp  3  SCR
549 AIR 1962 SC 1314 , SCR p. 557  When a question of law is fairly arguable, where there is
room for difference of opinion on it or where the Court thought it necessary to deal with that
question at some length and discuss alternative views, then the question would be a substantial
question of law.
On the other hand if the question was practically covered by the decision of the
highest court or if the general principles to be applied in determining the question are well settled
and the only question was of applying those principles to the particular fact of the case it would not
be a substantial question of law.
The proper test for determining whether a question of law raised in the case is substantial would
be, whether it is of general public importance or whether it directly and substantially affects the
rights of the parties and if so, whether it is either an open question in the sense that it is not finally
settled by this Court.
If the question is settled by the highest court or the general principles to be
applied in determining the question are well settled and there is a mere question of applying those
principles or the question raised is palpably absurd, the question would not be a substantial
question of law.
To be substantial, a question of law must be debatable, not previously settled by law of the land
or a binding precedent, and must have a material bearing on the decision of the case, if answered
either way, insofar as the rights of the parties before it are concerned.
To be a question of law
involving in the case there must be first, a foundation for it laid in the pleadings and the question
should emerge from the sustainable findings of fact arrived at by court of facts and it must be
necessary to decide that question of law for a just and proper decision of the case.
An entirely new
point raised for the first time before the High Court is not a question involved in the case unless it
goes to the root of the matter.
It will, therefore, depend on the facts and circumstance of each case
whether a question of law is a substantial one and involved in the case or not, the paramount overall
consideration being the need for striking a judicious balance between the indispensable obligation
to do justice at all stages and impelling necessity of avoiding prolongation in the life of any lis.
See
Santosh Hazari v. Purushottam Tiwari7 .
The principles relating to Section 100 of the CPC relevant for this case may be summarised thus

An inference of fact from the recitals or contents of a document is a question of
fact.
But the legal effect of the terms of a document is a question of law.
Construction
of a document involving the application of any principle of law, is also a question of
law.
Therefore, when there is misconstruction of a document or wrong 7  2001  3 SCC 179 application of
a principle of law in construing a document, it gives rise to a question of law.
The High Court should be satisfied that the case involves a substantial question of law, and not a
mere question of law.
A question of law having a material bearing on the decision of the case  that is, September, 2022
law, if it is not covered by any specific provisions of law or settled legal principle emerging from
binding precedents and involves a debatable legal issue.
A substantial question of law will also arise
in a contrary situation, where the legal position is clear, either on account of express provisions of
law or binding precedents, but the court below has decided the matter, either ignoring or acting
contrary to such legal principle.
In the second type of cases, the substantial question of law arises
not because the law is still debatable, but because the decision rendered on a material question,
violates the settled position of law.
The general rule is that the High Court will not interfere with findings of facts arrived at by the
courts below.
But it is not an absolute rule.
Some of the well-recognised exceptions are where  i  the
courts below have ignored material evidence or acted on no evidence  ii  the courts have drawn
wrong inferences from proved facts by applying the law erroneously or  iii  the courts have wrongly
cast the burden of proof.
When we refer to decision based on no evidence, it not only refers to
cases where there is a total dearth of evidence, but also refers to any case, where the evidence, taken
as a whole, is not reasonably capable of supporting the finding.
In this case, it cannot be said that the First Appellate Court acted on no evidence.
The
Respondents in their Second Appeal before the High Court did not advert to any material evidence
that had been ignored by the First Appellate Court.
The Respondents also could not show that any
wrong inference had been drawn by the First Appellate Court from proved facts by applying the law
erroneously.
In this case, as observed above, evidence had been adduced on behalf of the Original Plaintiff as
well as the Defendants.
The First Appellate Court analysed the evidence carefully and in effect found
that the Trial Court had erred in its analysis of evidence and given undue importance to
discrepancies and inconsistencies, which were not really material, overlooking the time gap of 34
years that had elapsed since the date of the adoption.
There was no such infirmity in the reasoning
of the First Appellate Court which called for interference.
Right of appeal is not automatic.
Right of appeal is conferred by statute.
When statute confers a
limited right of appeal restricted only to cases which involve substantial questions of law, it is not
open to this Court to sit in appeal over the factual findings arrived at by the First Appellate Court.
The questions raised in High Court, did not meet the tests laid down by this Court for holding
that the questions are substantial questions of law.
We are constrained to hold that there was no
question of law, let alone any substantial question of law, involved in the Second Appeal.
The appeal is, for the reasons, as discussed above, allowed.
The impugned judgment and order is
set aside and the judgment and order decree of the First Appellate Court in Regular Civil Appeal
No.361 of 1984 is restored.
INDIRA BANERJEE J. J.K. MAHESHWARI
NEW DELHI SEPTEMBER 22, 2022 September, 2022."""

texts_to_process =text3 ### you can also load your text files directly into this
data = Data(texts_to_process)  #### create Data object for data  preprocessing before running ML models
use_gpu = False
pipeline = Pipeline(components=['Rhetorical_Role'], use_gpu=use_gpu, verbose=True)

results = pipeline(data)

json_result_doc_1 = results[0]

ℹ Installing en_core_web_trf. This is a one time process!!
ℹ Pre-processing will happen on GPU!
ℹ Loading Rhetorical Role...
ℹ Rhetorical Roles will use CPU!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ℹ Preprocessing rhetorical role model input!!!


100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ℹ Processing documents with rhetorical role model!!!


100%|██████████| 1/1 [00:36<00:00, 36.27s/it]


In [ ]:
json_result_doc_1

{'id': 'a862da4731bab1ae635a7456653f0f52eb50c1ecbae15e924f59e5eece76d183',
 'data': {'text': '\nJudgment\nIndira Banerjee, J.\nLeave granted. \nThis appeal filed by the Original Plaintiff, Chandrabhan,  since deceased, represented by his legal\nrepresentatives , is against a final judgment and order dated 11  th January 2016 passed by the Aurangabad Bench of the High Court of Judicature at Bombay, allowing Second Appeal No.\n45 of 1995 filed by the Respondents, reversing the judgment and order dated 10th November 1994 passed by the Additional District Judge, Beed, in Regular Civil Appeal No.361 of 1984 and dismissing Regular Civil Suit No.\n198 of 1979 filed by the Original Plaintiff praying for the relief of declaration of ownership and perpetual injunction in respect of the suit property. \nSambhaji, grandfather of the Original Plaintiff, Chandrabhan had two sons, Baliram and Rambhau. \nBaliram was the elder of the two sons of Sambhaji. \nThe Original Defendant No.1 Yamunabai, was th